In [1]:
import requests, json
import pandas as pd
from pandas_ods_reader import read_ods

### Test Dataset: 
We prepared a small dataset with fact-checks and similar claims found on facebook or google. The Same-Context column is a first attempt to label wether the meaning between fact-check title and claim is the same. 

In [2]:
df = read_ods('example_list.ods', 1)
df

,Fact-Check,Claim,Same-Context
0,Leicester is going back into lockdown soon aft...,Strange Leicester is in lockdown just when the...,yes
1,The number of British children who need foster...,Experts say children in foster care could face...,no
2,"At mobile Covid-19 testing units, 97% of resul...",The UK’s mobile testing unit capacity is to be...,no
3,The last time a police officer was successfull...,Deaths in British police custody: no convicted...,yes
4,Tennis pro Serena Williams wrote a post saying...,"In a Facebook post, tennis star Serena William...",yes
5,Facebook has said sharing the Lord’s Prayer go...,Facebook says posting the Lord's prayer goes a...,yes
6,The Bill and Melinda Gates Foundation was kick...,"Don’t forget, the Gates family had to leave In...",yes
7,A little girl called Alice has been kidnapped ...,A little girl has vanished from a local area a...,yes
8,An illustration shows that the Covid-19 test i...,The test is done with a shallow nose swab not ...,no
9,"Bill Gates is a partner in the lab in Wuhan, C...",Bill Gates owns the lab where this all started,yes


The dataset is split into two lists: corpus and queries. Corpus contains all existing fact-checks and queries is the list of statements for which we want to find a match in the corpus list.

In [3]:
corpus = df['Fact-Check'].to_list()
queries = df['Claim'].dropna().to_list()

In [4]:
print(corpus[0])
print(queries[0])

Leicester is going back into lockdown soon after 5G was rolled out there. 
Strange Leicester is in lockdown just when they're rolling out 5G there coincidence or what


### API-Request
Both lists are sent to the /list_matching endpoint of Frankie's API. Since the API only works with json data it is important to set the Content-Type and format accordingly.

In [5]:
r = requests.post("http://api.frankie-ai.com/list_matching", 
                  data = json.dumps({
                        "corpus": corpus,
                        "queries": queries
                    }),
                  headers = {"Content-Type": "application/json"}
                 )

### Response
The response contains an array of found matches. The matches are sorted in the same order as the input query list. This allows to simply loop over all queries and select the found match using the same index as the current query. Apart of the matched fact-check title a match also provides a similarity score and the corpus index. This index can be used to select the row of the corpus list corresponding to the matched sentence.

In [6]:
response = r.json()
for idx in range(len(queries)):
    match = response[idx]
    querie = queries[idx]
    print("Querie: \n" + querie + "")
    print("Found Match: \n" + match['sentence'] + "")
    print("Similarity Score: " + str(match['score']) + "\n\n")
    
    # Alternative matched sentence selection:
    # print("Found Match: \n" + corpus[match['corpus_index']] + "")
    

Querie: 
Strange Leicester is in lockdown just when they're rolling out 5G there coincidence or what
Found Match: 
Leicester is going back into lockdown soon after 5G was rolled out there. 
Similarity Score: 72


Querie: 
Experts say children in foster care could face heightened risk of abuse, neglect amid pandemic
Found Match: 
The number of British children who need foster care has risen by 44% during the pandemic, children’s charity Barnardo’s has warned. 
Similarity Score: 44


Querie: 
The UK’s mobile testing unit capacity is to be boosted by the largest deployment of armed forces personnel during the Covid-19 outbreak.
Found Match: 
At mobile Covid-19 testing units, 97% of results are delivered within 24 hours. 
Similarity Score: 68


Querie: 
Deaths in British police custody: no convicted officers since 1969
Found Match: 
The last time a police officer was successfully prosecuted in the UK concerning the death of somebody in custody was in 1969. 
Similarity Score: 64


Querie: 
